MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Francesca Lazzeri Energy Demand Forecast Workbench workshop.

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Linear regression with recursive feature elimination

In [3]:
%matplotlib inline
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
from typing import Callable
# from azureml.core import Workspace, Dataset
# from azureml.core.experiment import Experiment
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


This notebook shows how to train a linear regression model to create a forecast of future energy demand. In particular, the model will be trained to predict energy demand in period $t_{+1}$, one hour ahead of the current time period $t$. This is known as 'one-step' time series forecasting because we are predicting one period into the future.

In [4]:
os.getcwd()

'/Users/brandon/Desktop/ccClub/ccClub 2021 Fall/Final_Project/ccclub-advanced-final/data-modeling'

In [5]:
WORKDIR = os.getcwd()
TRAIN_DIR = os.path.join(WORKDIR, '../data-processing/data/train')
TEST_DIR = os.path.join(WORKDIR, '../data-processing/data/test')
MODEL_NAME = "linear_regression"

In [6]:
# assign all the train data into DFs
for train_data in os.listdir(TRAIN_DIR):
    locals()['df_' + train_data[5:-4]] = pd.read_csv(TRAIN_DIR + '/' + train_data)
    print(f'The local variable of type DataFrame, df_{train_data[5:-4]} is created')

df_train_1.head()

The local variable of type DataFrame, df_train_4 is created
The local variable of type DataFrame, df_train_3 is created
The local variable of type DataFrame, df_train_2 is created
The local variable of type DataFrame, df_train_1 is created
The local variable of type DataFrame, df_outlier_train_4 is created
The local variable of type DataFrame, df_outlier_train_3 is created
The local variable of type DataFrame, df_outlier_train_2 is created
The local variable of type DataFrame, df_outlier_train_1 is created


,temperature,solar_ghi,solar_prediction_mw,wind_prediction_mw,load_actuals_mw,timestamp,month,season,workdayornot,temperature_lag1,...,wind_prediction_mw_lag3,wind_prediction_mw_lag4,wind_prediction_mw_lag5,wind_prediction_mw_lag6,solar_prediction_mw_lag1,solar_prediction_mw_lag2,solar_prediction_mw_lag3,solar_prediction_mw_lag4,solar_prediction_mw_lag5,solar_prediction_mw_lag6
0,134.6351,0.0,0.0,53.9495,89.6098,2020-01-01 02:30:00+01:00,1,1,0,134.6573,...,64.3057,66.9774,69.2968,70.8654,0.0,0.0,0.0,0.0,0.0,0.0
1,134.6129,0.0,0.0,50.5166,88.3333,2020-01-01 02:45:00+01:00,1,1,0,134.6351,...,61.1283,64.3057,66.9774,69.2968,0.0,0.0,0.0,0.0,0.0,0.0
2,134.5925,0.0,0.0,47.2634,87.3941,2020-01-01 03:00:00+01:00,1,1,0,134.6129,...,57.2304,61.1283,64.3057,66.9774,0.0,0.0,0.0,0.0,0.0,0.0
3,134.5740,0.0,0.0,44.7925,86.6259,2020-01-01 03:15:00+01:00,1,1,0,134.5925,...,53.9495,57.2304,61.1283,64.3057,0.0,0.0,0.0,0.0,0.0,0.0
4,134.5555,0.0,0.0,42.2637,86.3420,2020-01-01 03:30:00+01:00,1,1,0,134.5740,...,50.5166,53.9495,57.2304,61.1283,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# assign all the test data into DFs
for test_data in os.listdir(TEST_DIR):
    locals()['df_' + test_data[5:-4]] = pd.read_csv(TEST_DIR + '/' + test_data)
    print(f'The local variable of type DataFrame, df_{test_data[5:-4]} is created')
df_test_1.head()

The local variable of type DataFrame, df_test_4 is created
The local variable of type DataFrame, df_test_1 is created
The local variable of type DataFrame, df_test_2 is created
The local variable of type DataFrame, df_test_3 is created
The local variable of type DataFrame, df_outlier_test_4 is created
The local variable of type DataFrame, df_outlier_test_1 is created
The local variable of type DataFrame, df_outlier_test_3 is created
The local variable of type DataFrame, df_outlier_test_2 is created


,temperature,solar_ghi,solar_prediction_mw,wind_prediction_mw,load_actuals_mw,timestamp,month,season,workdayornot,temperature_lag1,...,wind_prediction_mw_lag3,wind_prediction_mw_lag4,wind_prediction_mw_lag5,wind_prediction_mw_lag6,solar_prediction_mw_lag1,solar_prediction_mw_lag2,solar_prediction_mw_lag3,solar_prediction_mw_lag4,solar_prediction_mw_lag5,solar_prediction_mw_lag6
0,142.5783,0.0,0.0,87.4705,79.7650,2020-06-01 00:00:00+02:00,6,3,0,142.6783,...,113.2723,116.0008,119.1572,122.0776,0.0,0.0,0.0,0.0,0.0,0.0
1,142.4826,0.0,0.0,85.4799,78.5278,2020-06-01 00:15:00+02:00,6,3,0,142.5783,...,112.6894,113.2723,116.0008,119.1572,0.0,0.0,0.0,0.0,0.0,0.0
2,142.3870,0.0,0.0,85.2095,77.1661,2020-06-01 00:30:00+02:00,6,3,0,142.4826,...,110.7785,112.6894,113.2723,116.0008,0.0,0.0,0.0,0.0,0.0,0.0
3,142.2914,0.0,0.0,86.4468,77.1635,2020-06-01 00:45:00+02:00,6,3,0,142.3870,...,87.4705,110.7785,112.6894,113.2723,0.0,0.0,0.0,0.0,0.0,0.0
4,142.1993,0.0,0.0,89.9961,77.1047,2020-06-01 01:00:00+02:00,6,3,0,142.2914,...,85.4799,87.4705,110.7785,112.6894,0.0,0.0,0.0,0.0,0.0,0.0


Create design matrix - each column in this matrix represents a model feature and each row is a training example. We remove the *demand* and *timeStamp* variables as they are not model features.

In [8]:
# Create a function that can retutn all the DataFrames of the training & testing variables
# making it easier to modify all the variables at once. This

# def modify_all_df(function: Callable) -> None:
#     for df_key in locals().keys():
#         if 'data_' in df_key:
#             # locals[df_key].apply(lambda x: function)
#             function(locals()[df_key])

In [9]:
import re
train_regex = re.compile(r'df_(\w+_)?train_\d')
train_variable_list = list(filter(train_regex.match, locals().keys()))
train_variable_list

['df_train_4',
 'df_train_3',
 'df_train_2',
 'df_train_1',
 'df_outlier_train_4',
 'df_outlier_train_3',
 'df_outlier_train_2',
 'df_outlier_train_1']

In [10]:
# set the X for every training data, by dropping the 'timestamp' & 'local_actual_mv' columns
for train_var in train_variable_list:
        locals()[f'X_{train_var[3:]}'] = locals()[train_var].drop(['timestamp', 'load_actuals_mw'], axis = 1)
        print(f'The varaible, X_{train_var[3:]} is created.')

        locals()[f'Y_{train_var[3:]}'] = locals()[train_var]['load_actuals_mw']
        print(f'The varaible, Y_{train_var[3:]} is created.')
        

The varaible, X_train_4 is created.
The varaible, Y_train_4 is created.
The varaible, X_train_3 is created.
The varaible, Y_train_3 is created.
The varaible, X_train_2 is created.
The varaible, Y_train_2 is created.
The varaible, X_train_1 is created.
The varaible, Y_train_1 is created.
The varaible, X_outlier_train_4 is created.
The varaible, Y_outlier_train_4 is created.
The varaible, X_outlier_train_3 is created.
The varaible, Y_outlier_train_3 is created.
The varaible, X_outlier_train_2 is created.
The varaible, Y_outlier_train_2 is created.
The varaible, X_outlier_train_1 is created.
The varaible, Y_outlier_train_1 is created.


In [11]:
# test_regex = re.compile(r'(\w+_)?test_\d')
# test_variable_list = list(filter(test_regex.match, locals().keys()))
# test_variable_list

In [12]:
# # set the Y for every training data, by selecting only the 'load_actuals_mw' columns
# for test_var in test_variable_list:
#         locals()[f'Y_{train_var}'] = locals()[train_var]['load_actuals_mw']
#         print(f'The varaible, Y_{train_var} is created.')

### Create predictive model pipeline

Here we use sklearn's Pipeline functionality to create a predictive model pipeline. For this model, the pipeline implements the following steps:
- **one-hot encode categorical variables** - this creates a feature for each unique value of a categorical feature. For example, the feature *dayofweek* has 7 unique values. This feature is split into 7 individual features dayofweek0, dayofweek1, ... , dayofweek6. The value of these features is 1 if the timeStamp corresponds to that day of the week, otherwise it is 0.
- **recursive feature elimination with cross validation (RFECV)** - it is often the case that some features add little predictive power to a model and may even make the model accuracy worse. Recursive feature elimination tests the model accuracy on increasingly smaller subsets of the features to identify the subset which produces the most accurate model. Cross validation is used to test each subset on multiple folds of the input data. The best model is that which achieves the lowest mean squared error averaged across the cross validation folds.
- **train final model** - the best model found in after the feature elimination process is used to train the final estimator on the whole dataset.

Identify indices for categorical columns for one hot encoding and create the OneHotEncoder:

In [13]:
dummy_cols = ['month', 'season', 'workdayornot']
sample_df = locals()["df_train_1"] # Cuz all the DFs' structure are the same, so we'll only need one sample for the col index
dummy_cols_index = [sample_df.columns.get_loc(col) for col in sample_df.columns if col in dummy_cols]
dummy_cols_index

[6, 7, 8]

In [14]:
preprocessor = ColumnTransformer([('encoder', OneHotEncoder(sparse=False), dummy_cols_index)], remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('encoder', OneHotEncoder(sparse=False),
                                 [6, 7, 8])])

Create the linear regression model object:

In [15]:
lr = LinearRegression(fit_intercept=True)
lr

LinearRegression()

For hyperparameter tuning and feature selection, cross validation will be performed using the training set. With time series forecasting, it is important that test data comes from a later time period than the training data. This also applies to each fold in cross validation. Therefore a time series split is used to create three folds for cross validation as illustrated below. Each time series plot represents a separate training/test split, with the training set coloured in blue and the test set coloured in red. Note that, even in the first split, the training data covers at least a full year so that the model can learn the annual seasonality of the demand.

In [16]:
tscv = TimeSeriesSplit(n_splits=3)

In [17]:
# demand_ts = df_train_1[['timestamp', 'load_actuals_mw']].copy()
# demand_ts.reset_index(drop=True, inplace=True)

# for split_num, split_idx  in enumerate(tscv.split(demand_ts)):
#     split_num = str(split_num)
#     train_idx = split_idx[0]
#     test_idx = split_idx[1]
#     demand_ts['fold' + split_num] = "not used"
#     demand_ts.loc[train_idx, 'fold' + split_num] = "train"
#     demand_ts.loc[test_idx, 'fold' + split_num] = "test"

# gs = gridspec.GridSpec(3,1)
# fig = plt.figure(figsize=(15, 10), tight_layout=True)

# ax = fig.add_subplot(gs[0])
# ax.plot(demand_ts.loc[demand_ts['fold0']=="train", "timestamp"], demand_ts.loc[demand_ts['fold0']=="train", "load_actuals_mw"], color='b')
# ax.plot(demand_ts.loc[demand_ts['fold0']=="test", "timestamp"], demand_ts.loc[demand_ts['fold0']=="test", "load_actuals_mw"], 'r')
# ax.plot(demand_ts.loc[demand_ts['fold0']=="not used", "timestamp"], demand_ts.loc[demand_ts['fold0']=="not used", "timestamp"], demand_ts.loc[demand_ts['fold0']=="not used", "load_actuals_mw"], 'w')

# ax = fig.add_subplot(gs[1], sharex=ax)
# plt.plot(demand_ts.loc[demand_ts['fold1']=="train", "timestamp"], demand_ts.loc[demand_ts['fold1']=="train", "load_actuals_mw"], 'b')
# plt.plot(demand_ts.loc[demand_ts['fold1']=="test", "timestamp"], demand_ts.loc[demand_ts['fold1']=="test", "load_actuals_mw"], 'r')
# plt.plot(demand_ts.loc[demand_ts['fold1']=="not used", "timestamp"], demand_ts.loc[demand_ts['fold1']=="not used", "load_actuals_mw"], 'w')

# ax = fig.add_subplot(gs[2], sharex=ax)
# plt.plot(demand_ts.loc[demand_ts['fold2']=="train", "timestamp"], demand_ts.loc[demand_ts['fold2']=="train", "load_actuals_mw"], 'b')
# plt.plot(demand_ts.loc[demand_ts['fold2']=="test", "timestamp"], demand_ts.loc[demand_ts['fold2']=="test", "load_actuals_mw"], 'r')
# plt.plot(demand_ts.loc[demand_ts['fold2']=="not used", "timestamp"], demand_ts.loc[demand_ts['fold2']=="not used", "load_actuals_mw"], 'w')
# plt.show()

Create the RFECV object. Note the metric for evaluating the model on each fold is the negative mean squared error. The best model is that which maximises this metric.

In [22]:
regr_cv = RFECV(estimator=lr,
             cv=tscv,
             scoring='neg_mean_squared_error',)

Create the model pipeline object.

In [23]:
lr_pipeline = Pipeline([('preprocessor', preprocessor), ('rfecv', regr_cv)])

Train the model pipeline. This should only take a few seconds.

In [24]:
lr_pipeline.fit(X_train_1, Y_train_1)

In [ ]:
# run.log("pipeline steps", regr_pipe.named_steps)

Save the trained model pipeline object.

In [ ]:
with open(os.path.join(WORKDIR, MODEL_NAME + '.pkl'), 'wb') as f:
    pickle.dump(lr_pipeline, f)

### Explore cross validation results

Best CV negative mean squared error:

In [ ]:
# run.log("best CV negMSE", max(regr_pipe.named_steps['rfecv'].grid_scores_))

Plot the cross validation errors with each subset of features. The chart shows that most features are useful to the model. However, the error gets significantly worse when there are 44 features or less in the model.

In [ ]:
cv_results = pd.DataFrame.from_dict({'cv_score': lr_pipeline.named_steps['rfecv'].grid_scores_})
cv_results['mean_squared_error'] = cv_results['cv_score']
plt.figure(figsize=(15, 5))
plt.plot(cv_results.index, cv_results['mean_squared_error'])
plt.xlabel('number of features')
plt.title('CV negative mean squared error')
# run.log_image("CV errors plot", plot=plt)
plt.show()


Number of features selected:

In [ ]:
lr_pipeline.named_steps['rfecv'].n_features_

Identify supported features after selection process:

In [ ]:
def get_onehot_cols(X):
    X_dummy_cols = list(pd.get_dummies(X.copy()[dummy_cols], columns=dummy_cols).columns)
    other_cols = list(X.columns.drop(dummy_cols))
    return X_dummy_cols + other_cols

supported_features = pd.DataFrame.from_dict(
    {'feature':get_onehot_cols(X), 
     'supported':lr_pipeline.named_steps['rfecv'].support_}
)
supported_features

Inspect model coefficients for each included feature. The magnitude and direction of the coefficients indicates the effect the features has on the model.

In [ ]:
coefs = supported_features.loc[supported_features['supported'], ].copy()
coefs['coefficients'] = lr_pipeline.named_steps['rfecv'].estimator_.coef_
coefs.plot.bar('feature', 'coefficients', figsize=(15, 3), legend=False)
# run.log_image("LR coefs per feature", plot=plt)
plt.show()

In [ ]:
# run.complete()